# Project Functionality

In [61]:
import pickle
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from ipywidgets import interact

In [42]:
supported_genres = {}

supported_genres['pop'] = pickle.load(open('model_cache/pop_model.p', 'rb'))
supported_genres['country'] = pickle.load(open('model_cache/country_model.p', 'rb'))
supported_genres['other'] = pickle.load(open('model_cache/other_model.p', 'rb'))

In [59]:
genre = input("What genre is your music? ")
while genre not in supported_genres:
    print(f"That genre is not currently supported. Please choose from {', '.join(supported_genres[:-1])}, or {supported_genres[-1]}.")
    genre = input("What genre is your music? ")

What genre is your music? other


In [56]:
date_string = input("Please enter the date that your song was released (MM-DD-YYYY): ")
release_date = datetime.strptime(date_string, '%m-%d-%Y')
# TODO: catch input errors

Please enter the date that your song was released (MM-DD-YYYY): 05-03-2020


In [51]:
def predictions(release_date, num_tweets=10, avg_char=40, t_rt_ratio=0.5, avg_freq=2):

    time_since_release = (datetime.now() - release_date).days // 7
    total_chars = num_tweets * avg_char
    num_t_plus_rt = (num_tweets / t_rt_ratio)
    num_rts = num_t_plus_rt - num_tweets

    X = [[time_since_release, num_tweets, total_chars, avg_char, num_rts, t_rt_ratio, avg_freq]]
    return(supported_genres[genre].predict(X)[0])

In [75]:
def make_plot(char = 40, ratio=0.5, freq = 2):
    x = []
    y = []
    for i in range(1, 11):
        num_tweets = i * 10
        x.append(num_tweets)
        y.append(predictions(release_date, num_tweets = num_tweets, avg_char = char, t_rt_ratio = ratio, avg_freq = freq))
    plt.scatter(x, y)
    
# TODO: Improve plot (use bokeh)

In [76]:
interact(make_plot, ratio = (0.1, 0.9), char = (20, 80, 10), freq = (0.5, 3, 0.5))
# TODO: Calculate global maximum across all parameters

interactive(children=(IntSlider(value=40, description='char', max=80, min=20, step=10), FloatSlider(value=0.5,…

<function __main__.make_plot(char=40, ratio=0.5, freq=2)>

# Project Description

Data Ingestion Sources:
- Rolling Stone Trending 25 Charts (built my own web scraper)
- Spotify API
- Twitter API

### Model Description

Features (for a single week data point):
- Number of original tweets by the user
- Number of retweets by the user
- Ratio of original tweets to retweets (tweets / (tweets + retweets))
- Average number of characters per original tweet
- Number of weeks since release of song
- Average frequency of original tweets by the user in units of tweets per day

In [77]:
# TODO: Complete project description

Genre selection: A word cloud representing the frequency of Spotify-provided genres across all artists in my dataset

![title](wordcloud.png)

In [ ]:
# TODO: Improve image (remove duplicates)